In [1]:
plt.style.use('bmh')

### Data preprocessing

#### Basic preprocessing

In [2]:
# Read dataset
df = pd.read_csv('data/Scotland EPC Dataset.csv')

# id of the dwelling as int
df['Property_UPRN'] = df['Property_UPRN'].astype(int)

# new date features
df['dt_day'] = pd.to_datetime(df['Date of Assessment'])
df['dt_month'] = df.dt_day.dt.month
del df['Date of Assessment']

# drop band features
feats = [
    'Current energy efficiency rating band',
    'Potential energy efficiency rating band',
    'Current Environmental Impact Rating Band',
    'Potential Environmental Impact Rating Band',
]
for f in feats:
    del df[f]
      
# Construction age converting to integer
dict_repl = {
 'before 1919': 1919,
 '1919-1929': 1929,
 '1930-1949': 1949,
 '1950-1964': 1964,
 '1965-1975': 1975,
 '1976-1983': 1983,
 '1984-1991': 1991,
 '1992-1998': 1998,
 '1999-2002': 2002,
 '2003-2007': 2007,
 '2008 onwards': 2008,
}
feat = 'Part 1 Construction Age Band'
df[feat] = df[feat].map(dict_repl)

# categorical features cleaning
feat = 'Built Form'
df[feat] = df[feat].fillna('').str.strip()
feat = 'Property Type'
df[feat] = df[feat].fillna('').str.strip()

#### Textual features describing the dwelling
1. Creating separate files with a flat structure - we will analyze them individually
2. Drop these features from initial dataset

In [3]:
# strip the text inside
feats = [
    "WALL_DESCRIPTION",
    "WALL_ENERGY_EFF",
    "ROOF_DESCRIPTION",
    "ROOF_ENERGY_EFF",
    "FLOOR_DESCRIPTION",
    "FLOOR_ENERGY_EFF",
    "FLOOR_ENV_EFF",
    "WINDOWS_DESCRIPTION",
    "WINDOWS_ENERGY_EFF",
    "WINDOWS_ENV_EFF",
    "MAINHEAT_DESCRIPTION",
    "MAINHEAT_ENERGY_EFF",
    "MAINHEAT_ENV_EFF",
    "MAINHEATCONT_DESCRIPTION",
    "MAINHEATC_ENERGY_EFF",
    "MAINHEATC_ENV_EFF",
    "HOT_WATER_ENERGY_EFF",
    "HOT_WATER_ENV_EFF",
    "LIGHTING_DESCRIPTION",
    "LIGHTING_ENERGY_EFF",
    "LIGHTING_ENV_EFF",
]
for feat in feats:
    df[feat] = df[feat].str.strip()
    
# drop duplicated features
feats = [
    'FLOOR_ENV_EFF',
    'WINDOWS_ENV_EFF',
    'MAINHEATC_ENV_EFF',
    'LIGHTING_ENV_EFF',
]
for feat in feats:
    del df[feat]

In [4]:
# transform textual features to the flat dataframe with dwelling id as the key
def transform_text(df1, feats):
    # 1) create lists from feature values
    for feat in feats:
        df1[feat] = df1[feat].map(lambda x: [x1.strip() for x1 in x.split('|')])
    # 2) transforming script
    out1 = list()
    out2 = list()
    _ = [out1.append([]) for l in feats]
    for _,r in df1.iterrows():
        for i,feat in enumerate(feats):
            out1[i]+=r[feat]
        l = [r['Property_UPRN']]*len(r[feat])
        out2+=l
    df_out = pd.DataFrame(out1).T
    df_out.columns = feats
    df_out['Property_UPRN'] = out2
    return df_out

In [5]:
feat1, feat2 = [
    "WALL_DESCRIPTION",
    "WALL_ENERGY_EFF",
]
# create flat dataframe
df1 = df[[feat1,feat2,'Property_UPRN']]
df_out = transform_text(df1,[feat1,feat2])
del df[feat1]
del df[feat2]
print(df_out.shape)
display(df_out[:5])
display(df_out.Property_UPRN.value_counts().value_counts())
df_out.to_csv(f'data/descriptions/{feat1}.csv', index = False, encoding = 'utf8', sep = '|')

(240029, 3)


,WALL_DESCRIPTION,WALL_ENERGY_EFF,Property_UPRN
0,"Cavity wall, as built, no insulation (assumed)",Poor,1001100841
1,"Cavity wall, filled cavity",Average,1001950838
2,"Cavity wall, as built, partial insulation (ass...",Average,1000995515
3,"Cavity wall, filled cavity",Average,1000995515
4,"Cavity wall, as built, insulated (assumed)",Good,1001256685


1    132483
2     50123
3      2432
4         1
Name: Property_UPRN, dtype: int64

In [6]:
feat1, feat2 = [
    "ROOF_DESCRIPTION",
    "ROOF_ENERGY_EFF",
]
# fill nulls
f1 = df[feat2] == 'N/A'
df.loc[f1,feat2] = np.nan
f1 = df[feat2].isnull()
df.loc[f1,feat1] = np.nan

# create flat dataframe
df1 = df[~df[feat2].isnull()][[feat1,feat2,'Property_UPRN']]
df_out = transform_text(df1,[feat1,feat2])
del df[feat1]
del df[feat2]
print(df_out.shape)
display(df_out[:5])
display(df_out.Property_UPRN.value_counts().value_counts())
df_out.to_csv(f'data/descriptions/{feat1}.csv', index = False, encoding = 'utf8', sep = '|')

(163285, 3)


,ROOF_DESCRIPTION,ROOF_ENERGY_EFF,Property_UPRN
0,"Pitched, 25 mm loft insulation",Poor,1001100841
1,"Pitched, insulated (assumed)",Average,1001950838
2,"Roof room(s), insulated (assumed)",Good,1001950838
3,"Roof room(s), no insulation (assumed)",Very Poor,1001950838
4,"Pitched, 250 mm loft insulation",Good,1001256685


1    107840
2     22660
3      3375
Name: Property_UPRN, dtype: int64

In [7]:
feat1, feat2 = [
    "FLOOR_DESCRIPTION",
    "FLOOR_ENERGY_EFF",
]
# FLOOR_ENERGY_EFF has a lot of nulls then we will drop it

# create flat dataframe
df1 = df[[feat1,feat2,'Property_UPRN']]
df_out = transform_text(df1,[feat1])
del df[feat1]
del df[feat2]

f1 = ~df_out.FLOOR_DESCRIPTION.isin(['(another dwelling below)','(other premises below)'])
df_out = df_out[f1]
print(df_out.shape)
display(df_out[:5])
display(df_out.Property_UPRN.value_counts().value_counts())
df_out.to_csv(f'data/descriptions/{feat1}.csv', index = False, encoding = 'utf8', sep = '|')

(145966, 2)


,FLOOR_DESCRIPTION,Property_UPRN
0,"Suspended, no insulation (assumed)",1001100841
1,"Suspended, no insulation (assumed)",1001950838
3,"Suspended, insulated (assumed)",1001256685
5,"Suspended, no insulation (assumed)",1000324880
6,"Suspended, no insulation (assumed)",1000902372


1    114726
2     14498
3       748
Name: Property_UPRN, dtype: int64

In [8]:
feat1, feat2 = [
    "WINDOWS_DESCRIPTION",
    "WINDOWS_ENERGY_EFF",
]
# create flat dataframe
df1 = df[[feat1,feat2,'Property_UPRN']]
df_out = transform_text(df1,[feat1,feat2])
del df[feat1]
del df[feat2]
print(df_out.shape)
display(df_out[:5])
display(df_out.Property_UPRN.value_counts().value_counts())
df_out.to_csv(f'data/descriptions/{feat1}.csv', index = False, encoding = 'utf8', sep = '|')

(185039, 3)


,WINDOWS_DESCRIPTION,WINDOWS_ENERGY_EFF,Property_UPRN
0,Description: Fully double glazed,Average,1001100841
1,Description: Fully double glazed,Average,1001950838
2,Description: Single glazed,Very Poor,1000995515
3,Description: Fully double glazed,Average,1001256685
4,Description: Fully double glazed,Good,1235709021


1    185039
Name: Property_UPRN, dtype: int64

In [9]:
feat1, feat2, feat3 = [
    "MAINHEAT_DESCRIPTION",
    "MAINHEAT_ENERGY_EFF",
    "MAINHEAT_ENV_EFF",
]
# create flat dataframe
df1 = df[[feat1,feat2,feat3,'Property_UPRN']]
df_out = transform_text(df1,[feat1,feat2,feat3])
del df[feat1]
del df[feat2]
del df[feat3]
print(df_out.shape)
display(df_out[:5])
display(df_out.Property_UPRN.value_counts().value_counts())
df_out.to_csv(f'data/descriptions/{feat1}.csv', index = False, encoding = 'utf8', sep = '|')

(186485, 4)


,MAINHEAT_DESCRIPTION,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,Property_UPRN
0,"Boiler and radiators, mains gas",Good,Good,1001100841
1,"Boiler and radiators, mains gas",Good,Good,1001950838
2,Electric storage heaters,Average,Very Poor,1000995515
3,"Boiler and radiators, mains gas",Good,Good,1001256685
4,"Boiler and radiators, mains gas",Good,Good,1235709021


1    183593
2      1446
Name: Property_UPRN, dtype: int64

In [10]:
feat1, feat2 = [
    "MAINHEATCONT_DESCRIPTION",
    "MAINHEATC_ENERGY_EFF",
]
# create flat dataframe
df1 = df[[feat1,feat2,'Property_UPRN']]
df_out = transform_text(df1,[feat1,feat2])
del df[feat1]
del df[feat2]
print(df_out.shape)
display(df_out[:5])
display(df_out.Property_UPRN.value_counts().value_counts())
df_out.to_csv(f'data/descriptions/{feat1}.csv', index = False, encoding = 'utf8', sep = '|')

(185954, 3)


,MAINHEATCONT_DESCRIPTION,MAINHEATC_ENERGY_EFF,Property_UPRN
0,"Programmer, room thermostat and TRVs",Good,1001100841
1,"Programmer, room thermostat and TRVs",Good,1001950838
2,Manual charge control,Poor,1000995515
3,"Programmer, room thermostat and TRVs",Good,1001256685
4,"Programmer, room thermostat and TRVs",Good,1235709021


1    184124
2       915
Name: Property_UPRN, dtype: int64

In [11]:
# hot-water replacement
feat1, feat2 = [
    "HOT_WATER_ENERGY_EFF",
    "HOT_WATER_ENV_EFF",
]
# it is one-dimensional feature
# replacement dictionary
dict_repl1 = {
    'very poor': 1,
    'poor': 2,
    'average': 3,
    'good': 4,
    'very good': 5,
}
df[feat1] = df[feat1].str.lower().map(dict_repl1)
print(df[feat1].isnull().sum())
df[feat2] = df[feat2].str.lower().map(dict_repl1)
print(df[feat2].isnull().sum())
df[feat2] = df[feat2].fillna(0).astype(int)

0
1


In [12]:
# lighting replacement
dict_repl2 = {
    'Low energy lighting in all fixed outlets': 100,
    'No low energy lighting': 0,
    'No Low energy lighting': 0,
}
l = len('Low energy lighting in ')
df['LIGHTING_DESCRIPTION'] = df.LIGHTING_DESCRIPTION.map(lambda x: dict_repl2[x] if x in dict_repl2 else x[l:].split('%')[0]).astype(int)
df['LIGHTING_ENERGY_EFF'] = df['LIGHTING_ENERGY_EFF'].str.lower().map(dict_repl1)
display(df.groupby('LIGHTING_ENERGY_EFF')['LIGHTING_DESCRIPTION'].agg([min,max,np.mean]))
# we can drop feature LIGHTING_ENV_EFF - it is just the function of LIGHTING_DESCRIPTION
del df['LIGHTING_ENERGY_EFF']

,min,max,mean
LIGHTING_ENERGY_EFF,,,
1,0,9,0.39
2,10,24,16.86
3,24,44,35.51
4,45,69,57.88
5,70,100,95.20


#### Merge towns with countries data
1. Get table of unique combinations of the Town and Postcode first part
2. Find possible countries defined by the postcode and defined by the town name
3. We define the final country by the following priority: by postcode, by town, other methods
4. Join this table with our initial table and export to file

In [13]:
# 1. Get table of unique combinations of the Town and Postcode first part
df['POST_TOWN'] = df['POST_TOWN'].fillna('').map(lambda x: x.upper().strip().replace('&AMP;','AND'))
df['Postcode2'] = df.Postcode.map(lambda x: x.split(' ')[0])
df_towns = df.groupby(['POST_TOWN','Postcode2']).Property_UPRN.count().reset_index()
print(df_towns.shape)
df_towns[-5:]

(1210, 3)


,POST_TOWN,Postcode2,Property_UPRN
1205,WHITBURN,EH47,14
1206,WICK,KW1,279
1207,WIGTOWN,DG8,2
1208,WINCHBURGH,EH52,31
1209,WISHAW,ML2,1247


In [14]:
# 2. Find possible countries defined by the postcode and defined by the town name
df_ag1 = pd.read_csv('data/metadata/towns.csv').groupby('Town')['Country'].agg(list)
df_ag2 = pd.read_csv('data/metadata/postcodes.csv').groupby('Postcode')['Country'].agg(list)

df_towns['c1'] = df_towns['POST_TOWN'].map(dict(df_ag1)).fillna('')
df_towns['c2'] = df_towns['Postcode2'].map(dict(df_ag2)).fillna('')
df_towns[-5:]

,POST_TOWN,Postcode2,Property_UPRN,c1,c2
1205,WHITBURN,EH47,14,[WEST LOTHIAN],[WEST LOTHIAN]
1206,WICK,KW1,279,[HIGHLAND],"[HIGHLAND, ORKNEY]"
1207,WIGTOWN,DG8,2,[DUMFRIES AND GALLOWAY],[DUMFRIES AND GALLOWAY]
1208,WINCHBURGH,EH52,31,[WEST LOTHIAN],[WEST LOTHIAN]
1209,WISHAW,ML2,1247,[NORTH LANARKSHIRE],[NORTH LANARKSHIRE]


In [15]:
# 3. We define the final country by the following priority: by postcode, by town, other methods.
def find_country(*args):
    # town, postcode, list1, list2
    t,p,a,b = args
    a = [] if len(a) == 0 else a
    b = [] if len(b) == 0 else b
    # here we can definitely choose the country
    if len(a) == 1:
        return a[0]
    if len(b) == 1:
        return b[0]
    # if we have the town name in the country list let's say that it is the country 
    if t in a:
        return t
    if t in b:
        return t
    # we have about 5k of towns without exact definition
    # let's give them the most popular value or random from the possible list
    s = pd.Series(a+b).value_counts()
    if len(s) > 0:
        return s.index[0]
    # manual replacing
    if t == "BO'NESS":
        return 'FALKIRK'
    if p == 'G34':
        return 'GLASGOW'
    # these postcodes from here: http://www.monikie.org.uk/scottish-postcodes.html
    if p in ['AB10','AB11','AB12','AB13','AB14','AB15','AB16','AB21','AB22','AB23','AB24','AB25']:
        return 'ABERDEEN'
    return ''

df_towns['Country'] = df_towns.apply(lambda x: find_country(x.POST_TOWN,x.Postcode2,x.c1,x.c2), axis = 1)
del df_towns['Property_UPRN']
del df_towns['c1']
del df_towns['c2']

df_towns[:3]

,POST_TOWN,Postcode2,Country
0,,AB10,ABERDEEN
1,,AB11,ABERDEEN
2,,AB14,ABERDEEN


In [16]:
# 4. Join this table with our initial table and export to file
print(df.shape)
df = df.merge(df_towns, how = 'left')
df['Country'].fillna('', inplace = True)
del df['Postcode2']
df.to_csv('data/data_clean.csv', index = False, encoding = 'utf8')
print(df.shape)
df[:3]

(185039, 28)
(185039, 28)


,Property_UPRN,Postcode,POST_TOWN,Primary Energy Indicator (kWh/m²/year),Total floor area (m²),Current energy efficiency rating,Potential Energy Efficiency Rating,Current Environmental Impact Rating,Potential Environmental Impact Rating,CO2 Emissions Current Per Floor Area (kg.CO2/m²/yr),HOT_WATER_ENERGY_EFF,HOT_WATER_ENV_EFF,LIGHTING_DESCRIPTION,Current Emissions (T.CO2/yr),Potential Reduction in Emissions (T.CO2/yr),Total current energy costs over 3 years (£),Current heating costs over 3 years (£),Potential heating costs over 3 years (£),Current hot water costs over 3 years (£),Potential hot water costs over 3 years (£),Current lighting costs over 3 years (£),Potential lighting costs over 3 years (£),Part 1 Construction Age Band,Built Form,Property Type,dt_day,dt_month,Country
0,1001100841,EH4 5EZ,EDINBURGH,375.00,94.00,53.00,85.00,45.00,82.00,66.00,2,2,100,6.20,4.20,"3,789.00","2,922.00","1,548.00",645.00,219.00,222.00,222.00,"1,949.00",Semi-Detached,House,2021-01-01,1,EDINBURGH
1,1001950838,EH7 4HE,EDINBURGH,250.00,175.00,66.00,80.00,57.00,75.00,44.00,4,4,100,7.70,2.80,"4,635.00","4,068.00","3,015.00",246.00,246.00,321.00,321.00,"1,929.00",End-Terrace,House,2021-01-01,1,EDINBURGH
2,1000995515,EH4 2DL,EDINBURGH,403.00,72.00,61.00,78.00,48.00,64.00,68.00,1,2,50,4.90,1.60,"3,570.00","2,226.00","1,191.00","1,038.00",564.00,306.00,207.00,"1,975.00",Semi-Detached,Flat,2021-02-01,2,EDINBURGH
